In [7]:
# Import libraries
import pandas as pd
import nltk
import nltk
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters 
n = 5 #the # of article headlines displayed per ticker
tickers = ['AAPL', 'TSLA', 'AMZN','NIO']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



Recent News Headlines for AAPL: 
Best Dow Jones Stocks To Buy And Watch In February 2021: Apple, Nike Near New Buy Points ( Feb-11-21 09:36AM )
Netflix co-founder: Disney emulating' its streaming playbook: 'Were seeing new rules of the game' ( 09:22AM )
Apple (AAPL) App Store Targeted by North Dakota Lawmakers ( 08:59AM )
Climate change will cause industry disruptions at an unprecedented rate: Gro Intelligence founder ( 08:50AM )
Is Google's New Privacy Push Another Nail in the Coffin for Facebook? ( 07:00AM )


Recent News Headlines for TSLA: 
SkyBridge's Scaramucci puts $100,000 price target on Bitcoin, sees more money managers and CFOs adopting it ( Feb-11-21 09:41AM )
Dow Jones Futures Signal Market Rally As Tilray, Climax Stocks Aren't Finished, Bitcoin Hits Record High; The Bull Case For Tesla ( 08:24AM )
Why Companies Like Peloton Are Issuing Convertible Debt on Attractive Terms ( 07:00AM )
Audi Challenges Tesla With New Electric Sedan E-Tron GT ( 06:34AM )
The Zacks Analyst B